# ABOUT


Datascientest's Datascientist continuous bootcamp - cohorte Mars2022 -  AeroBOT project

**Tutor**

* Alban THUET

**Authors:**

* Hichem HADJI  

</br>

---
</br>

**Version History**

Version | Date       | Author(s)  | Modification
--------|----------- | ---------  | --------------------------
1.0     | 20/10/2022 | H.H        | document creation

This notebook can be executed entirely. 

It

* mounts the GDrive of our AeroBot project @gmail account.

* loads the data from the `df_for_Anomaly_prediction.pkl` under a pandas DataFrame named `df`, which contains `96986` entries and 20 columns.
These data do not contain any more UAS-related entries.

* Perform a modeling using a One VS All approach while focusing on 2 shallow machine learning models (Decision Tree & Random Forest) and using an unsupervised feature selection 

* Perform a modeling usung a One VS All while using an undersampeling. 

# IMPORT PACKAGES


settings for  full / patial Narrative display. Helene?


In [ ]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns

#######################
# Pandas
#######################
import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option("display.colheader_justify","left") # left-justify the print output of pandas

### Display full columnwidth
# Set pandas settings to display full text columns
#pd.options.display.max_colwidth = None
# Restore pandas settings to display standard colwidth
pd.reset_option('display.max_colwidth')

import itertools # Pour créer des iterateurs

# Package to show the progression of pandas operations
from tqdm import tqdm
# from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()
# simply use .progress_apply() instead of .apply() on your pd.DataFram

######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# Define global plot parameters for better readability and consistency among plots
# A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 23
plt.rcParams['xtick.labelsize'] = 23
plt.rcParams['ytick.labelsize'] = 23
plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus
from nltk.stem.snowball import EnglishStemmer

##############################
# Undersampeling
##############################
from imblearn.under_sampling import RandomUnderSampler 

###############################
# ML preprocessing and models
###############################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import ensemble # random forest
from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix

import pickle as pkl # Saving data externally

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# LOAD DATA

## Mount GDrive

In [ ]:
#@title
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

#check your present working directory 
%pwd

Mounted at /content/drive/


'/content'

In [ ]:
#@title
# move to the transformed data location (you can create a deeper structure, if needed, e.g. to save a trained model):
%cd /content/drive/MyDrive/data/transformed/

/content/drive/MyDrive/data/transformed


In [ ]:
#@title
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

 2022_09_11_7_4_3_raw_narr_BERT_BASE_frozen_max_length_345.pkl
 complaints-2022-08-05_13_55.csv
'Copy of Qualified abbreviations_20220718.xlsx.gsheet'
'Data Dictionnary.xlsx'
 data_for_BERT_multilabel_20220805.pkl
 df_for_Anomaly_prediction.pkl
 model.png
 model_results
 Narrative_PP_stemmed_24072022_TRAIN.pkl
 Narrative_Raw_Stemmed_24072022_TRAIN.pkl
 Narrative_RegEx_subst_21072022_TRAIN.pkl
'Qualified abbreviations_20220707_test.csv'
'Qualified abbreviations_20220708.csv'
'Qualified abbreviations_20220718.csv'
'Qualified abbreviations_20220718_Google_sheet.gsheet'
 test_data_final.pkl
 train_data_final.pkl


## Load data from .pkl file


In [ ]:
# Load the TRAIN data (97417 entries)
# Do not touch the TEST data until the end of the project!
# or the curse of the greek gods will fall upon you!

%cd /content/drive/MyDrive/data/transformed/
with open("df_for_Anomaly_prediction.pkl", "rb") as f:
    loaded_data = pkl.load(f)

df = loaded_data
print("\nA Dataframe with", len(df), "entries has been loaded")

/content/drive/MyDrive/data/transformed

A Dataframe with 96986 entries has been loaded


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96986 entries, 1163382 to 874642
Data columns (total 20 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   Narrative                                     96986 non-null  object
 1   Anomaly                                       96986 non-null  object
 2   Narrative_PP_stemmed                          96986 non-null  object
 3   Narrative_Raw_Stemmed                         96986 non-null  object
 4   Narrative_Raw_Stemmed_str                     96986 non-null  object
 5   Narrative_PP_stemmed_str                      96986 non-null  object
 6   Anomaly_Deviation / Discrepancy - Procedural  96986 non-null  int64 
 7   Anomaly_Aircraft Equipment                    96986 non-null  int64 
 8   Anomaly_Conflict                              96986 non-null  int64 
 9   Anomaly_Inflight Event / Encounter            96986 non-null  int

In [ ]:
# Retriece the list of Anomaly label columns
Anomaly_RootLabels_columns = []

for col in df.columns:
  if 'Anomaly_' in str(col):
      Anomaly_RootLabels_columns.append(col)

In [ ]:
Anomaly_RootLabels_columns

['Anomaly_Deviation / Discrepancy - Procedural',
 'Anomaly_Aircraft Equipment',
 'Anomaly_Conflict',
 'Anomaly_Inflight Event / Encounter',
 'Anomaly_ATC Issue',
 'Anomaly_Deviation - Altitude',
 'Anomaly_Deviation - Track / Heading',
 'Anomaly_Ground Event / Encounter',
 'Anomaly_Flight Deck / Cabin / Aircraft Event',
 'Anomaly_Ground Incursion',
 'Anomaly_Airspace Violation',
 'Anomaly_Deviation - Speed',
 'Anomaly_Ground Excursion',
 'Anomaly_No Specific Anomaly Occurred']

# PREDICTION

## DECISION TREE (Gini , Max_Depth=20), RAW NARRATIVES,DEFAULT SAMPLING  

In [ ]:
def metrics_table(data, target, vectorizer, classifier, x_vec):
    """
    Return Prediction metrics and feature importance for Decision Trees
    /!\ works only for DT, since the other models don't return Feature importance

    Inputs: 
    - data: pd.Series containing the narratives as single string
    - target: a list of target labels
    - vectorizer instance. Lets you instantiate it with the desired settings
    - classifier (must be a decision tree)
    
    Return:
    - a DataFrame containing the classification reports for each target label
    - a DataFrame containing the feature importance for each token

    """

    # Time the function execution
    import time
    start_time = time.time()
    print(7*'-', "Execution started...", 7*'-', '\n')
    
    # Train-test split  
    X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                        test_size= 0.2, 
                                                        random_state = 12)


    # apply the vectorizer 
    X_train = vectorizer.fit_transform(X_train) 
    X_test = vectorizer.transform(X_test) 
    vocab = vectorizer.get_feature_names_out()
    print("Vocabulary length:" , len(vocab))

    # Instantiate variables
    df_importances = pd.DataFrame(vocab).rename(columns={0: 'token'})
    
    reports = []
    # Loop through Anomaly labels
    for Anomaly in target.columns:

      print('Prediction of anomaly :', Anomaly)

      # Select the correct target data
      y_train_Anomaly = y_train[Anomaly]
      y_test_Anomaly = y_test[Anomaly]
     
      # Train & predict
      classifier.fit(X_train, y_train_Anomaly)
      y_pred = classifier.predict(X_test)

      #########  FEATURE IMPORTANCE ################
      # write feature and its importance into a dict
      feats={}
      for feature,importance in zip(vocab, classifier.feature_importances_):
        feats[feature] = importance

      # Create a DataFrame from the dict. Each anomaly label gets a new column
      col_importance = 'Importance_' + Anomaly
      importance = pd.DataFrame.from_dict(feats, orient = 'index').reset_index().rename(columns = {'index': 'token', 0: col_importance})
      df_importances = df_importances.merge(importance, on = 'token', how = 'left')

      #########  CLASSIFICATION REPORT #############
      # Append the classification report into a list
      #print(confusion_matrix(y_test, y_pred))
      report = classification_report(y_test_Anomaly, y_pred, output_dict =True)
      reports.append(report)

    # Fill in the rows with the anomaly label
    anomaly_name = pd.DataFrame(np.repeat(target.columns,4), columns = ['anomaly'])
    classifier_name = pd.DataFrame(np.repeat(clf.__class__.__name__,56), columns = ['classifier'])
    narr_preprosseing = pd.DataFrame(np.repeat(0,56), columns = ['preprocessing'])
    undersampling = pd.DataFrame(np.repeat(0,56), columns = ['undersampling'])
    count_vectorizer_opts  = pd.DataFrame(np.repeat(x_vec,56), columns = ['Count_Vectorizer_opts'])
    

    d = pd.DataFrame()
    for data in reports:
      report = pd.DataFrame(data)
      d = pd.concat([d, report])

    d = d.reset_index()
    d = d.rename(columns ={'index' :'metric'})
    df_reports = pd.concat([anomaly_name,classifier_name,narr_preprosseing,undersampling,count_vectorizer_opts, d], axis=1)

    # Calculate and print time to execute
    end_time = time.time()
    print(f"--- DONE! It took {np.round((end_time - start_time)/60,1)} minutes ---")

    return df_reports, df_importances

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_Raw_Stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer() 

In [ ]:
# Call the function
df_metrics_DT_RAW, df_importances_DT_RAW = metrics_table(data, target, vectorizer, clf, x_vec=np.nan)

------- Execution started... ------- 

Vocabulary length: 51652
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 4.9 minutes ---


In [ ]:
df_metrics_DT_RAW

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,0,0,NaN,precision,0.641804,0.706571,0.686875,0.674187,0.680705
1,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,0,0,NaN,recall,0.488705,0.818642,0.686875,0.653674,0.686875
2,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,0,0,NaN,f1-score,0.554888,0.758489,0.686875,0.656688,0.677177
3,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,0,0,NaN,support,7747.000000,11651.000000,0.686875,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,DecisionTreeClassifier,0,0,NaN,precision,0.843562,0.824463,0.836736,0.834012,0.836014
5,Anomaly_Aircraft Equipment,DecisionTreeClassifier,0,0,NaN,recall,0.896267,0.745630,0.836736,0.820948,0.836736
6,Anomaly_Aircraft Equipment,DecisionTreeClassifier,0,0,NaN,f1-score,0.869116,0.783067,0.836736,0.826092,0.835110
7,Anomaly_Aircraft Equipment,DecisionTreeClassifier,0,0,NaN,support,11732.000000,7666.000000,0.836736,19398.000000,19398.000000
8,Anomaly_Conflict,DecisionTreeClassifier,0,0,NaN,precision,0.911975,0.690803,0.878750,0.801389,0.872479
9,Anomaly_Conflict,DecisionTreeClassifier,0,0,NaN,recall,0.943454,0.581120,0.878750,0.762287,0.878750


In [ ]:
import pickle
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_RAW.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_RAW], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## PREDICTION  DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x DEFAULT SAMPLING  

In [ ]:
def metrics_table_PP(data, target, vectorizer, classifier, x_vec):
    """
    Return Prediction metrics and feature importance for Decision Trees
    /!\ works only for DT, since the other models don't return Feature importance

    Inputs: 
    - data: pd.Series containing the narratives as single string
    - target: a list of target labels
    - vectorizer instance. Lets you instantiate it with the desired settings
    - classifier (must be a decision tree)
    
    Return:
    - a DataFrame containing the classification reports for each target label
    - a DataFrame containing the feature importance for each token

    """

    # Time the function execution
    import time
    start_time = time.time()
    print(7*'-', "Execution started...", 7*'-', '\n')
    
    # Train-test split  
    X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                        test_size= 0.2, 
                                                        random_state = 12)


    # apply the vectorizer 
    X_train = vectorizer.fit_transform(X_train) 
    X_test = vectorizer.transform(X_test) 
    vocab = vectorizer.get_feature_names_out()
    print("Vocabulary length:" , len(vocab))

    # Instantiate variables
    df_importances = pd.DataFrame(vocab).rename(columns={0: 'token'})
    reports = []

    # Loop through Anomaly labels
    for Anomaly in target.columns:

      print('Prediction of anomaly :', Anomaly)

      # Select the correct target data
      y_train_Anomaly = y_train[Anomaly]
      y_test_Anomaly = y_test[Anomaly]
     
      # Train & predict
      classifier.fit(X_train, y_train_Anomaly)
      y_pred = classifier.predict(X_test)

      #########  FEATURE IMPORTANCE ################
      # write feature and its importance into a dict
      feats={}
      for feature,importance in zip(vocab, classifier.feature_importances_):
        feats[feature] = importance

      # Create a DataFrame from the dict. Each anomaly label gets a new column
      col_importance = 'Importance_' + Anomaly
      importance = pd.DataFrame.from_dict(feats, orient = 'index').reset_index().rename(columns = {'index': 'token', 0: col_importance})
      df_importances = df_importances.merge(importance, on = 'token', how = 'left')

      #########  CLASSIFICATION REPORT #############
      # Append the classification report into a list
      #print(confusion_matrix(y_test, y_pred))
      report = classification_report(y_test_Anomaly, y_pred, output_dict =True)
      reports.append(report)

    # Fill in the rows with the anomaly label
    anomaly_name = pd.DataFrame(np.repeat(target.columns,4), columns = ['anomaly'])
    classifier_name = pd.DataFrame(np.repeat(clf.__class__.__name__,56), columns = ['classifier'])
    narr_preprosseing = pd.DataFrame(np.repeat(1,56), columns = ['preprocessing'])
    undersampling = pd.DataFrame(np.repeat(0,56), columns = ['undersampling'])
    count_vectorizer_opts  = pd.DataFrame(np.repeat(x_vec,56), columns = ['Count_Vectorizer_opts'])
    
    
    d = pd.DataFrame()
    for data in reports:
      report = pd.DataFrame(data)
      d = pd.concat([d, report])

    d = d.reset_index()
    d = d.rename(columns ={'index' :'metric'})
    df_reports = pd.concat([anomaly_name,classifier_name,narr_preprosseing,undersampling,count_vectorizer_opts, d], axis=1)

    # Calculate and print time to execute
    end_time = time.time()
    print(f"--- DONE! It took {np.round((end_time - start_time)/60,1)} minutes ---")

    return df_reports, df_importances

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer() 

### Perform a decision tree model on the different target and stock the results

In [ ]:
df_metrics_DT_PP, df_importances_DT_PP = metrics_table_PP(data, target, vectorizer, clf, x_vec = np.nan)

------- Execution started... ------- 

Vocabulary length: 50030
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 5.0 minutes ---


In [ ]:
df_metrics_DT_PP

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,NaN,precision,0.643865,0.701661,0.684761,0.672763,0.678579
1,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,NaN,recall,0.471408,0.826624,0.684761,0.649016,0.684761
2,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,NaN,f1-score,0.544303,0.759034,0.684761,0.651668,0.673276
3,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,NaN,support,7747.000000,11651.000000,0.684761,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,NaN,precision,0.845555,0.814993,0.834416,0.830274,0.833477
5,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,NaN,recall,0.888510,0.751631,0.834416,0.820070,0.834416
6,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,NaN,f1-score,0.866500,0.782030,0.834416,0.824265,0.833118
7,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,NaN,support,11732.000000,7666.000000,0.834416,19398.000000,19398.000000
8,Anomaly_Conflict,DecisionTreeClassifier,1,0,NaN,precision,0.914460,0.685505,0.878957,0.799983,0.873574
9,Anomaly_Conflict,DecisionTreeClassifier,1,0,NaN,recall,0.940630,0.595266,0.878957,0.767948,0.878957


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_PP], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## PREDICTION  DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x REMOVE Numbers

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b")

In [ ]:
df_metrics_DT_PP_RN, df_importances_DT_PP_RN = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers')

------- Execution started... ------- 

Vocabulary length: 36543
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 4.7 minutes ---


In [ ]:
df_metrics_DT_PP_RN

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers,precision,0.643107,0.702456,0.684968,0.672781,0.678754
1,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers,recall,0.474506,0.824908,0.684968,0.649707,0.684968
2,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers,f1-score,0.546089,0.758773,0.684968,0.652431,0.673833
3,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers,support,7747.000000,11651.000000,0.684968,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers,precision,0.846796,0.817346,0.836066,0.832071,0.835158
5,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers,recall,0.889959,0.753587,0.836066,0.821773,0.836066
6,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers,f1-score,0.867841,0.784173,0.836066,0.826007,0.834776
7,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers,support,11732.000000,7666.000000,0.836066,19398.000000,19398.000000
8,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers,precision,0.914615,0.689967,0.879988,0.802291,0.874498
9,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers,recall,0.941823,0.595554,0.879988,0.768688,0.879988


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_RN.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_PP_RN], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


# Min_df =12

## PREDICTION DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x REMOVE Numbers x min_df_12

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = DecisionTreeClassifier(max_depth = 20)

vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 12)

In [ ]:
df_metrics_DT_PP_RN_MDF_12, df_importances_DT_PP_RN_MDF_12 = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers; min_df = 12')

------- Execution started... ------- 

Vocabulary length: 9140
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 4.5 minutes ---


In [ ]:
df_metrics_DT_PP_RN_MDF_12

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,precision,0.640224,0.701426,0.683369,0.670825,0.676984
1,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,recall,0.472957,0.823277,0.683369,0.648117,0.683369
2,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,f1-score,0.544024,0.757482,0.683369,0.650753,0.672233
3,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,support,7747.000000,11651.000000,0.683369,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,precision,0.847354,0.815786,0.835808,0.831570,0.834879
5,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,recall,0.888595,0.755022,0.835808,0.821809,0.835808
6,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,f1-score,0.867485,0.784229,0.835808,0.825857,0.834582
7,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,support,11732.000000,7666.000000,0.835808,19398.000000,19398.000000
8,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,precision,0.913600,0.685198,0.878441,0.799399,0.872813
9,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 12,recall,0.941007,0.590647,0.878441,0.765827,0.878441


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_RN_MDF_12.pkl'
#enregistre le modèle
pkl.dump([df_metrics_DT_PP_RN_MDF_12], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## Random Forest

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 12)

In [ ]:
df_metrics_RF_PP_RN_MDF_12, df_importances_RF_PP_RN_MDF_12 = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers; min_df = 12')

------- Execution started... ------- 

Vocabulary length: 9140
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 23.3 minutes ---


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df_metrics_RF_PP_RN_MDF_12

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,precision,0.765464,0.743019,0.749304,0.754241,0.751983
1,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,recall,0.536724,0.890653,0.749304,0.713689,0.749304
2,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,f1-score,0.631004,0.810165,0.749304,0.720584,0.738613
3,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,support,7747.000000,11651.000000,0.749304,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,precision,0.884784,0.882095,0.883802,0.883440,0.883722
5,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,recall,0.928827,0.814897,0.883802,0.871862,0.883802
6,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,f1-score,0.906271,0.847166,0.883802,0.876718,0.882913
7,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,support,11732.000000,7666.000000,0.883802,19398.000000,19398.000000
8,Anomaly_Conflict,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,precision,0.900430,0.884281,0.898804,0.892355,0.897546
9,Anomaly_Conflict,RandomForestClassifier,1,0,exclude_numbers; min_df = 12,recall,0.985816,0.498557,0.898804,0.742187,0.898804


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_RF_PP_RN_MDF_12.pkl'
#enregistre le modèle
pkl.dump([df_metrics_RF_PP_RN_MDF_12], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


# Min_df =100

## PREDICTION DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x REMOVE Numbers x min_df_100

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 100)

In [ ]:
df_metrics_DT_PP_RN_MDF_100, df_importances_DT_PP_RN_MDF_100 = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers; min_df = 100')

------- Execution started... ------- 

Vocabulary length: 3683
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 4.4 minutes ---


In [ ]:
df_metrics_DT_PP_RN_MDF_100

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,precision,0.641430,0.701505,0.683833,0.671467,0.677513
1,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,recall,0.472441,0.824393,0.683833,0.648417,0.683833
2,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,f1-score,0.544117,0.758000,0.683833,0.651058,0.672581
3,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,support,7747.000000,11651.000000,0.683833,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,precision,0.846516,0.816575,0.835602,0.831545,0.834683
5,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,recall,0.889448,0.753196,0.835602,0.821322,0.835602
6,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,f1-score,0.867451,0.783606,0.835602,0.825528,0.834316
7,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,support,11732.000000,7666.000000,0.835602,19398.000000,19398.000000
8,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,precision,0.913236,0.695014,0.880297,0.804125,0.874267
9,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 100,recall,0.943956,0.587471,0.880297,0.765714,0.880297


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_RN_MDF_100.pkl'
#enregistre le modèle
pkl.dump([df_metrics_DT_PP_RN_MDF_100], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## Random Forest

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 100)

In [ ]:
df_metrics_RF_PP_RN_MDF_100, df_importances_RF_PP_RN_MDF_100 = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers; min_df = 100')

------- Execution started... ------- 

Vocabulary length: 3683
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 26.8 minutes ---


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df_metrics_RF_PP_RN_MDF_100

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,precision,0.763416,0.745282,0.750438,0.754349,0.752524
1,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,recall,0.543565,0.887992,0.750438,0.715779,0.750438
2,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,f1-score,0.635000,0.810402,0.750438,0.722701,0.740352
3,Anomaly_Deviation / Discrepancy - Procedural,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,support,7747.000000,11651.000000,0.750438,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,precision,0.889171,0.879950,0.885761,0.884560,0.885527
5,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,recall,0.926611,0.823245,0.885761,0.874928,0.885761
6,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,f1-score,0.907505,0.850654,0.885761,0.879079,0.885038
7,Anomaly_Aircraft Equipment,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,support,11732.000000,7666.000000,0.885761,19398.000000,19398.000000
8,Anomaly_Conflict,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,precision,0.907464,0.876468,0.904062,0.891966,0.901929
9,Anomaly_Conflict,RandomForestClassifier,1,0,exclude_numbers; min_df = 100,recall,0.983494,0.538684,0.904062,0.761089,0.904062


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_RF_PP_RN_MDF_100.pkl'
#enregistre le modèle
pkl.dump([df_metrics_RF_PP_RN_MDF_100], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


# Min_df =1600

## PREDICTION DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x REMOVE Numbers x min_df_1600

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 1600)

In [ ]:
df_metrics_DT_PP_RN_MDF_1600, df_importances_DT_PP_RN_MDF_1600 = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers; min_df = 1600')

------- Execution started... ------- 

Vocabulary length: 858
Prediction of anomaly : Anomaly_Deviation / Discrepancy - Procedural
Prediction of anomaly : Anomaly_Aircraft Equipment
Prediction of anomaly : Anomaly_Conflict
Prediction of anomaly : Anomaly_Inflight Event / Encounter
Prediction of anomaly : Anomaly_ATC Issue
Prediction of anomaly : Anomaly_Deviation - Altitude
Prediction of anomaly : Anomaly_Deviation - Track / Heading
Prediction of anomaly : Anomaly_Ground Event / Encounter
Prediction of anomaly : Anomaly_Flight Deck / Cabin / Aircraft Event
Prediction of anomaly : Anomaly_Ground Incursion
Prediction of anomaly : Anomaly_Airspace Violation
Prediction of anomaly : Anomaly_Deviation - Speed
Prediction of anomaly : Anomaly_Ground Excursion
Prediction of anomaly : Anomaly_No Specific Anomaly Occurred
--- DONE! It took 4.1 minutes ---


In [ ]:
df_metrics_DT_PP_RN_MDF_1600

,anomaly,classifier,preprocessing,undersampling,Count_Vectorizer_opts,metric,0,1,accuracy,macro avg,weighted avg
0,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,precision,0.639196,0.707254,0.686308,0.673225,0.680073
1,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,recall,0.492578,0.815123,0.686308,0.653850,0.686308
2,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,f1-score,0.556390,0.757367,0.686308,0.656878,0.677102
3,Anomaly_Deviation / Discrepancy - Procedural,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,support,7747.000000,11651.000000,0.686308,19398.000000,19398.000000
4,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,precision,0.842896,0.814636,0.832663,0.828766,0.831728
5,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,recall,0.889021,0.746413,0.832663,0.817717,0.832663
6,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,f1-score,0.865345,0.779033,0.832663,0.822189,0.831235
7,Anomaly_Aircraft Equipment,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,support,11732.000000,7666.000000,0.832663,19398.000000,19398.000000
8,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,precision,0.911812,0.643038,0.868028,0.777425,0.863815
9,Anomaly_Conflict,DecisionTreeClassifier,1,0,exclude_numbers; min_df = 1600,recall,0.929208,0.586605,0.868028,0.757907,0.868028


In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_RN_MDF_1600.pkl'
#enregistre le modèle
pkl.dump([df_metrics_DT_PP_RN_MDF_1600], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## Random Forest

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
target = df[Anomaly_RootLabels_columns]
clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 1600)

In [ ]:
df_metrics_RF_PP_RN_MDF_1600, df_importances_RF_PP_RN_MDF_1600 = metrics_table_PP(data, target, vectorizer, clf, x_vec = 'exclude_numbers; min_df = 1600')

In [ ]:
df_metrics_RF_PP_RN_MDF_1600

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_RF_PP_RN_MDF_1600.pkl'
#enregistre le modèle
pkl.dump([df_metrics_RF_PP_RN_MDF_1600], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


# PREDICTION UNDERSAMPELING

## DECISION TREE (Gini , Max_Depth=20), RAW NARRATIVES,DEFAULT SAMPLING  

In [ ]:
def metrics_table_US(data, target, vectorizer, classifier, x_vec):
    """
    Return Prediction metrics and feature importance for Decision Trees
    /!\ works only for DT, since the other models don't return Feature importance

    Inputs: 
    - data: pd.Series containing the narratives as single string
    - target: a list of target labels
    - vectorizer instance. Lets you instantiate it with the desired settings
    - classifier (must be a decision tree)
    
    Return:
    - a DataFrame containing the classification reports for each target label
    - a DataFrame containing the feature importance for each token

    """

    # Time the function execution
    import time
    start_time = time.time()
    print(7*'-', "Execution started...", 7*'-', '\n')
    
    # add an extra column ones in the data DataFrame for convenience later during undersampeling
    data = pd.DataFrame(data)
    data['ones'] = 1

    # Train-test split  
    X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                        test_size= 0.2, 
                                                        random_state = 12)

    # Perform an undersampeling step
    rus = RandomUnderSampler()
    X_ru, y_ru = rus.fit_resample(X_train, y_train)

    # Remove the extra added column of ones from the X_ru and X_test 
    # Remove from X_ru
    X_ru = X_ru.drop('ones', axis=1)
    X_ru = X_ru.squeeze()

    # Remove from X_test
    X_test = X_test.drop('ones', axis=1)
    X_test = X_test.squeeze()


    # apply the vectorizer 
    X_ru = vectorizer.fit_transform(X_ru) 
    X_test = vectorizer.transform(X_test) 
    vocab = vectorizer.get_feature_names_out()
    print("Vocabulary length:" , len(vocab))

  

    # Instantiate variables
    df_importances = pd.DataFrame(vocab).rename(columns={0: 'token'})
    reports = []

    # Loop through Anomaly labels
    for Anomaly in target.columns:

      print('Prediction of anomaly :', Anomaly)

      # Select the correct target data
      y_ru_Anomaly = y_ru[Anomaly]
      y_test_Anomaly = y_test[Anomaly]
     
      # Train & predict
      classifier.fit(X_ru, y_ru_Anomaly)
      y_pred = classifier.predict(X_test)

      #########  FEATURE IMPORTANCE ################
      # write feature and its importance into a dict
      feats={}
      for feature,importance in zip(vocab, classifier.feature_importances_):
        feats[feature] = importance

      # Create a DataFrame from the dict. Each anomaly label gets a new column
      col_importance = 'Importance_' + Anomaly
      importance = pd.DataFrame.from_dict(feats, orient = 'index').reset_index().rename(columns = {'index': 'token', 0: col_importance})
      df_importances = df_importances.merge(importance, on = 'token', how = 'left')

      #########  CLASSIFICATION REPORT #############
      # Append the classification report into a list
      #print(confusion_matrix(y_test, y_pred))
      report = classification_report(y_test_Anomaly, y_pred, output_dict =True)
      reports.append(report)

    # Fill in the rows with the anomaly label
    anomaly_name = pd.DataFrame(np.repeat(target.columns,4), columns = ['anomaly'])
    classifier_name = pd.DataFrame(np.repeat(clf.__class__.__name__,4), columns = ['classifier'])
    narr_preprosseing = pd.DataFrame(np.repeat(0,4), columns = ['preprocessing'])
    undersampling = pd.DataFrame(np.repeat(1,4), columns = ['undersampling'])
    count_vectorizer_opts  = pd.DataFrame(np.repeat(x_vec,4), columns = ['Count_Vectorizer_opts'])
    
    d = pd.DataFrame()

    for data in reports:
      report = pd.DataFrame(data)
      d = pd.concat([d, report])

    d = d.reset_index()
    d = d.rename(columns ={'index' :'metric'})
    df_reports = pd.concat([anomaly_name,classifier_name,narr_preprosseing,undersampling,count_vectorizer_opts, d], axis=1)

    # Calculate and print time to execute
    end_time = time.time()
    print(f"--- DONE! It took {np.round((end_time - start_time)/60,1)} minutes ---")

    return df_reports, df_importances

In [ ]:
data = df['Narrative_Raw_Stemmed_str']
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer() 

In [ ]:
# Call the function
df_metrics_DT_RAW_US = pd.DataFrame()
df_importances_DT_RAW_US = pd.DataFrame()
for element in Anomaly_RootLabels_columns:
  target = pd.DataFrame(df[element])
  df_metric, df_importance = metrics_table_US(data, target, vectorizer, clf)
  df_metrics_DT_RAW_US = pd.concat([df_metrics_DT_RAW_US, df_metric])
  df_importances_DT_RAW_US = pd.concat([df_importances_DT_RAW_US, df_importance])

In [ ]:
df_metrics_DT_RAW_US

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_RAW_US.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_RAW_US], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## PREDICTION  DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x DEFAULT SAMPLING  

In [ ]:
def metrics_table_PP_US(data, target, vectorizer, classifier, x_vec):
    """
    Return Prediction metrics and feature importance for Decision Trees
    /!\ works only for DT, since the other models don't return Feature importance

    Inputs: 
    - data: pd.Series containing the narratives as single string
    - target: a list of target labels
    - vectorizer instance. Lets you instantiate it with the desired settings
    - classifier (must be a decision tree)
    
    Return:
    - a DataFrame containing the classification reports for each target label
    - a DataFrame containing the feature importance for each token

    """

    # Time the function execution
    import time
    start_time = time.time()
    print(7*'-', "Execution started...", 7*'-', '\n')
    
    # add an extra column ones in the data DataFrame for convenience later during undersampeling
    data = pd.DataFrame(data)
    data['ones'] = 1

    # Train-test split  
    X_train, X_test, y_train, y_test = train_test_split(data, target, 
                                                        test_size= 0.2, 
                                                        random_state = 12)

    # Perform an undersampeling step
    rus = RandomUnderSampler()
    X_ru, y_ru = rus.fit_resample(X_train, y_train)

    # Remove the extra added column of ones from the X_ru and X_test 
    # Remove from X_ru
    X_ru = X_ru.drop('ones', axis=1)
    X_ru = X_ru.squeeze()

    # Remove from X_test
    X_test = X_test.drop('ones', axis=1)
    X_test = X_test.squeeze()


    # apply the vectorizer 
    X_ru = vectorizer.fit_transform(X_ru) 
    X_test = vectorizer.transform(X_test) 
    vocab = vectorizer.get_feature_names_out()
    print("Vocabulary length:" , len(vocab))

  

    # Instantiate variables
    df_importances = pd.DataFrame(vocab).rename(columns={0: 'token'})
    reports = []

    # Loop through Anomaly labels
    for Anomaly in target.columns:

      print('Prediction of anomaly :', Anomaly)

      # Select the correct target data
      y_ru_Anomaly = y_ru[Anomaly]
      y_test_Anomaly = y_test[Anomaly]
     
      # Train & predict
      classifier.fit(X_ru, y_ru_Anomaly)
      y_pred = classifier.predict(X_test)

      #########  FEATURE IMPORTANCE ################
      # write feature and its importance into a dict
      feats={}
      for feature,importance in zip(vocab, classifier.feature_importances_):
        feats[feature] = importance

      # Create a DataFrame from the dict. Each anomaly label gets a new column
      col_importance = 'Importance_' + Anomaly
      importance = pd.DataFrame.from_dict(feats, orient = 'index').reset_index().rename(columns = {'index': 'token', 0: col_importance})
      df_importances = df_importances.merge(importance, on = 'token', how = 'left')

      #########  CLASSIFICATION REPORT #############
      # Append the classification report into a list
      #print(confusion_matrix(y_test, y_pred))
      report = classification_report(y_test_Anomaly, y_pred, output_dict =True)
      reports.append(report)

    # Fill in the rows with the anomaly label
    anomaly_name = pd.DataFrame(np.repeat(target.columns,4), columns = ['anomaly'])
    classifier_name = pd.DataFrame(np.repeat(clf.__class__.__name__,4), columns = ['classifier'])
    narr_preprosseing = pd.DataFrame(np.repeat(1,4), columns = ['preprocessing'])
    undersampling = pd.DataFrame(np.repeat(1,4), columns = ['undersampling'])
    count_vectorizer_opts  = pd.DataFrame(np.repeat(x_vec,4), columns = ['Count_Vectorizer_opts'])
    
    d = pd.DataFrame()

    for data in reports:
      report = pd.DataFrame(data)
      d = pd.concat([d, report])

    d = d.reset_index()
    d = d.rename(columns ={'index' :'metric'})
    df_reports = pd.concat([anomaly_name,classifier_name,narr_preprosseing,undersampling,count_vectorizer_opts, d], axis=1)

    # Calculate and print time to execute
    end_time = time.time()
    print(f"--- DONE! It took {np.round((end_time - start_time)/60,1)} minutes ---")

    return df_reports, df_importances

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer() 

In [ ]:
# Call the function
df_metrics_DT_PP_US = pd.DataFrame()
df_importances_PP_US = pd.DataFrame()
for element in Anomaly_RootLabels_columns:
  target = pd.DataFrame(df[element])
  df_metric, df_importance = metrics_table_PP_US(data, target, vectorizer, clf, x_vec =np.nan)
  df_metrics_DT_PP_US = pd.concat([df_metrics_DT_PP_US, df_metric])
  df_importances_PP_US = pd.concat([df_importances_PP_US, df_importance])

In [ ]:
df_metrics_DT_PP_US

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_US.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_PP_US], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## PREDICTION  DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x REMOVE Numbers

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b")

In [ ]:
# Call the function
df_metrics_DT_PP_US_RN = pd.DataFrame()
df_importances_PP_US_RN = pd.DataFrame()
for element in Anomaly_RootLabels_columns:
  target = pd.DataFrame(df[element])
  df_metric, df_importance = metrics_table_PP_US(data, target, vectorizer, clf, x_vec ='exclude_numbers')
  df_metrics_DT_PP_US_RN = pd.concat([df_metrics_DT_PP_US_RN, df_metric])
  df_importances_PP_US_RN = pd.concat([df_importances_PP_US_RN, df_importance])

In [ ]:
df_metrics_DT_PP_US_RN

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_US_RN.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_PP_US_RN], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## PREDICTION DECISION TREE (Gini , Max_Depth=20) x PP NARRATIVES x REMOVE Numbers x min_df setting

In [ ]:
# initialize the objects necesseray function inputs
data = df['Narrative_PP_stemmed_str']
clf = DecisionTreeClassifier(max_depth = 20)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 12)

In [ ]:
# Call the function
df_metrics_DT_PP_US_RN_MDF = pd.DataFrame()
df_importances_PP_US_RN_MDF = pd.DataFrame()
for element in Anomaly_RootLabels_columns:
  target = pd.DataFrame(df[element])
  df_metric, df_importance = metrics_table_PP_US(data, target, vectorizer, clf, x_vec ='exclude_numbers; min_df = 12')
  df_metrics_DT_PP_US_RN_MDF = pd.concat([df_metrics_DT_PP_US_RN_MDF, df_metric])
  df_importances_PP_US_RN_MDF = pd.concat([df_importances_PP_US_RN_MDF, df_importance])

In [ ]:
df_metrics_DT_PP_US_RN_MDF

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_DT_PP_US_RN_MDF.pkl'
#enregistre le modèle
pickle.dump([df_metrics_DT_PP_US_RN_MDF], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results


## Random Forest

In [ ]:
data = df['Narrative_PP_stemmed_str']
clf = ensemble.RandomForestClassifier(n_jobs = -1, random_state = 321)
vectorizer = CountVectorizer(token_pattern=r"\b[a-zA-Z][a-zA-Z]+\b",
                             min_df = 12)

In [ ]:
# Call the function
df_metrics_RF_PP_US_RN_MDF = pd.DataFrame()
df_importances_RF_US_RN_MDF = pd.DataFrame()
for element in Anomaly_RootLabels_columns:
  target = pd.DataFrame(df[element])
  df_metric, df_importance = metrics_table_PP_US(data, target, vectorizer, clf, x_vec ='exclude_numbers; min_df = 12')
  df_metrics_RF_PP_US_RN_MDF = pd.concat([df_metrics_RF_PP_US_RN_MDF, df_metric])
  df_importances_RF_US_RN_MDF = pd.concat([df_importances_RF_US_RN_MDF, df_importance])

In [ ]:
df_metrics_RF_PP_US_RN_MDF

In [ ]:
%cd /content/drive/MyDrive/data/transformed/model_results/
filename = 'df_metrics_RF_PP_US_RN_MDF.pkl'
#enregistre le modèle
pickle.dump([df_metrics_RF_PP_US_RN_MDF], open(filename, 'wb'))

/content/drive/MyDrive/data/transformed/model_results
